In [2]:
import pandas as pd
import sys
import os
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
sys.path.insert(0, os.path.abspath('../')) 
from lib.common import constants as k
from lib.common import gra
from lib.eirgrid import data as eirgrid_data
from lib.common.marginal_emissions import compute_mef

In [3]:
fuel_mix = pd.read_csv(k.PROCESSED_DATA_DIR / 'fuel_mix.csv', index_col=0)
fuel_mix.rename(columns={'StartTime': 'EffectiveTime'}, inplace=True)
fuel_mix['EffectiveTime'] = pd.to_datetime(fuel_mix['EffectiveTime'])
                                           
pricing  = pd.read_csv(k.RAW_DATA_DIR / 'semo' / 'price_all_latest.csv', index_col=0)
pricing = pricing[['StartTime', 'NetImbalanceVolume', 'ImbalanceSettlementPrice']]
pricing.rename(columns={'StartTime': 'EffectiveTime'}, inplace=True)
pricing['EffectiveTime'] = pd.to_datetime(pricing['EffectiveTime'])

eirgrid = compute_mef(eirgrid_data.system()).dropna().replace([np.inf, -np.inf], np.nan).ffill()

In [4]:
combined_sys = eirgrid.merge(fuel_mix, on='EffectiveTime', how='left')
combined_sys = combined_sys.merge(pricing, on='EffectiveTime', how='left')
combined_sys.shape, combined_sys.dtypes

((119997, 24),
 EffectiveTime                datetime64[ns]
 SysFrequency                        float64
 Co2Emissions                        float64
 Co2Intensity                        float64
 SystemDemand                        float64
 GenExp                              float64
 InterNet                            float64
 WindActual                          float64
 MarginalEmissions                   float64
 Fuel_Battery_Storage                float64
 Fuel_Biomass_Peat                   float64
 Fuel_Coal                           float64
 Fuel_Distillate                     float64
 Fuel_Gas                            float64
 Fuel_Oil                            float64
 Fuel_Other_Non_Renewable            float64
 Fuel_Other_Renewable                float64
 Fuel_Pumped_Storage_Hydro           float64
 Fuel_Renewable_Hydro                float64
 Fuel_Solar                          float64
 Fuel_Waste                          float64
 Fuel_Wind                           flo

In [5]:
def score(excluded_features=[]):
  df = gra.dt_encode(combined_sys.drop(columns=excluded_features), 'EffectiveTime')
  features = df.copy()
  scaler = MinMaxScaler()
  scaler.fit(features)
  features[features.columns] = scaler.transform(features[features.columns])
  target = features['MarginalEmissions']
  features = features.drop(columns = 'MarginalEmissions')

  grg = gra.grg(features, target)
  display(features.shape)
  return grg.mean()

Scores without,
* Pricing information
* Frequency information
* CI information

---

In [6]:
score(excluded_features=[
  'SysFrequency',
  'Co2Intensity',
  'Co2Emissions',
  'ImbalanceSettlementPrice',
  'NetImbalanceVolume',
])

(57716, 19)

Fuel_Other_Renewable         0.785356
SystemDemand                 0.777216
InterNet                     0.777019
GenExp                       0.774400
Fuel_Gas                     0.764736
Fuel_Other_Non_Renewable     0.734765
Fuel_Renewable_Hydro         0.704294
WindActual                   0.682820
Fuel_Wind                    0.679740
Fuel_Coal                    0.676386
EffectiveTime_cos            0.636831
EffectiveTime_sin            0.636365
Fuel_Waste                   0.636206
Fuel_Biomass_Peat            0.629097
Fuel_Oil                     0.561280
Fuel_Pumped_Storage_Hydro    0.549927
Fuel_Solar                   0.516363
Fuel_Distillate              0.506322
Fuel_Battery_Storage         0.495445
dtype: float64

Scores without,
* Pricing information

In [22]:
score(excluded_features=[
  'ImbalanceSettlementPrice',
  'NetImbalanceVolume',
])

(57716, 22)

SysFrequency                 0.837926
Fuel_Other_Renewable         0.785356
SystemDemand                 0.777216
InterNet                     0.777019
GenExp                       0.774400
Fuel_Gas                     0.764736
Co2Intensity                 0.761515
Fuel_Other_Non_Renewable     0.734765
Fuel_Renewable_Hydro         0.704294
Co2Emissions                 0.683564
WindActual                   0.682820
Fuel_Wind                    0.679740
Fuel_Coal                    0.676386
EffectiveTime_cos            0.636831
EffectiveTime_sin            0.636365
Fuel_Waste                   0.636206
Fuel_Biomass_Peat            0.629097
Fuel_Oil                     0.561280
Fuel_Pumped_Storage_Hydro    0.549927
Fuel_Solar                   0.516363
Fuel_Distillate              0.506322
Fuel_Battery_Storage         0.495445
dtype: float64

Scores without,
* Carbon intensity
* System frequency

*NB: The data set is reduced from nearly 50,000 points to only 1469 (limited by pricing information)

In [24]:
score(excluded_features=[
  'SysFrequency',
  'Co2Intensity',
])

(1469, 22)

Fuel_Other_Non_Renewable     0.880841
NetImbalanceVolume           0.824185
ImbalanceSettlementPrice     0.787317
GenExp                       0.732600
Fuel_Renewable_Hydro         0.715458
SystemDemand                 0.710221
Fuel_Other_Renewable         0.707512
Co2Emissions                 0.703498
Fuel_Gas                     0.702300
InterNet                     0.677364
WindActual                   0.667211
Fuel_Wind                    0.666148
EffectiveTime_sin            0.636399
EffectiveTime_cos            0.635727
Fuel_Biomass_Peat            0.605787
Fuel_Waste                   0.578203
Fuel_Oil                     0.568564
Fuel_Solar                   0.565729
Fuel_Coal                    0.551945
Fuel_Pumped_Storage_Hydro    0.536869
Fuel_Battery_Storage         0.492990
Fuel_Distillate              0.486650
dtype: float64

Score with all fields

In [25]:
score()

(1469, 24)

Fuel_Other_Non_Renewable     0.880841
NetImbalanceVolume           0.824185
ImbalanceSettlementPrice     0.787317
SysFrequency                 0.785514
Co2Intensity                 0.760462
GenExp                       0.732600
Fuel_Renewable_Hydro         0.715458
SystemDemand                 0.710221
Fuel_Other_Renewable         0.707512
Co2Emissions                 0.703498
Fuel_Gas                     0.702300
InterNet                     0.677364
WindActual                   0.667211
Fuel_Wind                    0.666148
EffectiveTime_sin            0.636399
EffectiveTime_cos            0.635727
Fuel_Biomass_Peat            0.605787
Fuel_Waste                   0.578203
Fuel_Oil                     0.568564
Fuel_Solar                   0.565729
Fuel_Coal                    0.551945
Fuel_Pumped_Storage_Hydro    0.536869
Fuel_Battery_Storage         0.492990
Fuel_Distillate              0.486650
dtype: float64